### Readme

## Make a data subdirectory and put all .hgt files there BEFORE continuing 

In [ ]:
import rasterio
from rasterio.merge import merge
from rasterio.transform import array_bounds
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium.raster_layers import ImageOverlay
import io
import base64
import pandas as pd

# ---------------------------------------
# Step 1: Load and Merge HGT Elevation Files
# ---------------------------------------
files = [
    "data/n33w118.hgt", "data/n33w119.hgt", "data/n33w120.hgt",
    "data/n34w118.hgt", "data/n34w119.hgt", "data/n34w120.hgt"
]
src_files_to_mosaic = [rasterio.open(fp) for fp in files]
mosaic, out_trans = merge(src_files_to_mosaic)
src_crs = src_files_to_mosaic[0].crs

# ---------------------------------------
# Step 2: Downsample the Elevation Data
# ---------------------------------------
factor = 5
downsampled = mosaic[:, ::factor, ::factor]
elev = downsampled[0]  # Select elevation band

# Update transform for the downsampled image
new_transform = out_trans * out_trans.scale(factor, factor)

# ---------------------------------------
# Step 3: Normalize and Export Elevation as PNG
# ---------------------------------------
normed = (elev - np.nanmin(elev)) / (np.nanmax(elev) - np.nanmin(elev))
image_8bit = (normed * 255).astype(np.uint8)

# Save image as PNG
plt.imsave("downsampled_elevation.png", image_8bit, cmap='terrain')

# Optional: Also create a base64 version for embedding
buf = io.BytesIO()
plt.imsave(buf, image_8bit, cmap='terrain')
buf.seek(0)
image_data = base64.b64encode(buf.read()).decode('utf-8')
data_url = 'data:image/png;base64,' + image_data

# ---------------------------------------
# Step 4: Export Elevation Data to CSV
# ---------------------------------------
# Flatten for export; optionally you could include lat/lon if desired
pd.DataFrame(elev).to_csv("downsampled_elevation.csv", index=False)

# ---------------------------------------
# Step 5: Display on Folium Map
# ---------------------------------------
rows, cols = downsampled.shape[1], downsampled.shape[2]
bounds = array_bounds(rows, cols, new_transform)
folium_bounds = [[bounds[1], bounds[0]], [bounds[3], bounds[2]]]

center_lat = (folium_bounds[0][0] + folium_bounds[1][0]) / 2
center_lon = (folium_bounds[0][1] + folium_bounds[1][1]) / 2
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add overlay (refers to saved PNG)
overlay = ImageOverlay(
    name="Elevation",
    image="downsampled_elevation.png",
    bounds=folium_bounds,
    opacity=0.6,
    interactive=True,
    cross_origin=False,
    zindex=1,
)
overlay.add_to(m)
folium.LayerControl().add_to(m)

# Save the map
m.save("elevation_map.html")
